In [34]:
import numpy as np
import pandas as pd
import chardet
import datetime
from datetime import timedelta
import os
from pathlib import Path

## Read File

In [35]:
# Detect the encoding of the file
with open('newleads.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Read the file using the detected encoding
df = pd.read_csv('newleads.csv', encoding=result['encoding'])

In [36]:
df.head()

,ApplicantId,Filed As,Status,CreatedOn,Source,Feedback Score,Country,Registration #,Registration Type,Registration Expiry,Employment Preference Attributes,Specialty,Seniority,Primary Consultant,Primary Email
0,223002,Hannah Mccleave,New Lead,18/07/2024 23:48,Organic Search,NaN,United Kingdom,NaN,NaN,NaN,Permanent,"GP Emergency, GP Anaesthetics",Consultant / Specialist,NaN,NaN
1,223016,Abdikadir Dahir,New Lead,19/07/2024 18:29,Organic Search,NaN,Somalia,NaN,NaN,NaN,Permanent,Surgery - General,Consultant / Specialist,NaN,NaN
2,223017,Malik El Issa,New Lead,19/07/2024 22:05,Organic Search,NaN,Switzerland,NaN,NaN,NaN,"Permanent, Telehealth",Radiology - General,Consultant / Specialist,NaN,NaN
3,223018,Isabella Baker-Whalley,New Lead,20/07/2024 19:23,Other,NaN,Australia,NaN,NaN,NaN,Permanent,Unknown,Unknown,NaN,NaN
4,223019,Anil Pandey,New Lead,20/07/2024 21:36,Organic Search,NaN,Nepal,NaN,NaN,NaN,Permanent,Paeds - General,Consultant / Specialist,NaN,NaN


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4347 entries, 0 to 4346
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ApplicantId                       4347 non-null   int64  
 1   Filed As                          4347 non-null   object 
 2   Status                            4347 non-null   object 
 3   CreatedOn                         4347 non-null   object 
 4   Source                            4347 non-null   object 
 5   Feedback Score                    0 non-null      float64
 6   Country                           4346 non-null   object 
 7   Registration #                    987 non-null    object 
 8   Registration Type                 985 non-null    object 
 9   Registration Expiry               967 non-null    object 
 10  Employment Preference Attributes  4347 non-null   object 
 11  Specialty                         4345 non-null   object 
 12  Senior

## Remove Test

In [38]:
# Define substrings to filter
substrings_to_remove = ['test', 'graham long', 'lizzie new' , 'Tahnee Love']

# Create a boolean mask to filter rows
mask = ~df['Filed As'].str.lower().str.contains('|'.join(substrings_to_remove), case=False)

# Apply the mask to the DataFrame
df = df[mask]

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4337 entries, 0 to 4346
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ApplicantId                       4337 non-null   int64  
 1   Filed As                          4337 non-null   object 
 2   Status                            4337 non-null   object 
 3   CreatedOn                         4337 non-null   object 
 4   Source                            4337 non-null   object 
 5   Feedback Score                    0 non-null      float64
 6   Country                           4336 non-null   object 
 7   Registration #                    987 non-null    object 
 8   Registration Type                 985 non-null    object 
 9   Registration Expiry               967 non-null    object 
 10  Employment Preference Attributes  4337 non-null   object 
 11  Specialty                         4335 non-null   object 
 12  Seniority  

In [40]:
#remove RMS
# List of consultants to remove
consultants_to_remove = ['Leiara Ferrett', 'Ebony Kouka', 'Alexandra Graham', 'Lisa Greaves', 'Elinor Faulkner']

# Filter out rows where Primary Consultant is in the list of consultants to remove
df = df[~df['Primary Consultant'].isin(consultants_to_remove)]

# remove JB LinkedIn Connection
df = df[df['Source'] != 'JB LinkedIn Connection']

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3531 entries, 0 to 4342
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ApplicantId                       3531 non-null   int64  
 1   Filed As                          3531 non-null   object 
 2   Status                            3531 non-null   object 
 3   CreatedOn                         3531 non-null   object 
 4   Source                            3531 non-null   object 
 5   Feedback Score                    0 non-null      float64
 6   Country                           3530 non-null   object 
 7   Registration #                    983 non-null    object 
 8   Registration Type                 981 non-null    object 
 9   Registration Expiry               963 non-null    object 
 10  Employment Preference Attributes  3531 non-null   object 
 11  Specialty                         3529 non-null   object 
 12  Seniority  

In [42]:
# Filter condition to remove specific emails from 'Primary Email'
condition = (df['Primary Email'].str.endswith('@wave.com.au')) & (~df['Primary Email'].isin(['unknown@wave.com.au', 'tbc@wave.com.au']))

# Removing rows based on the condition
df = df[~condition]

# Dropping the 'Primary Email' column
df = df.drop('Primary Email', axis=1)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3526 entries, 0 to 4342
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ApplicantId                       3526 non-null   int64  
 1   Filed As                          3526 non-null   object 
 2   Status                            3526 non-null   object 
 3   CreatedOn                         3526 non-null   object 
 4   Source                            3526 non-null   object 
 5   Feedback Score                    0 non-null      float64
 6   Country                           3526 non-null   object 
 7   Registration #                    983 non-null    object 
 8   Registration Type                 981 non-null    object 
 9   Registration Expiry               963 non-null    object 
 10  Employment Preference Attributes  3526 non-null   object 
 11  Specialty                         3524 non-null   object 
 12  Seniority  

In [44]:
# Create a boolean mask to filter rows
mask = ~((df['Status'] == 'Imported') & (df['Source'].isin(['Headhunt', 'Linkedin'])))

# Apply the mask to the original DataFrame 'df'
df = df[mask]

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3425 entries, 0 to 4342
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ApplicantId                       3425 non-null   int64  
 1   Filed As                          3425 non-null   object 
 2   Status                            3425 non-null   object 
 3   CreatedOn                         3425 non-null   object 
 4   Source                            3425 non-null   object 
 5   Feedback Score                    0 non-null      float64
 6   Country                           3425 non-null   object 
 7   Registration #                    898 non-null    object 
 8   Registration Type                 896 non-null    object 
 9   Registration Expiry               878 non-null    object 
 10  Employment Preference Attributes  3425 non-null   object 
 11  Specialty                         3423 non-null   object 
 12  Seniority  

## add week number

In [46]:
# Function to calculate the week number based on weeks ending on Sundays
def calculate_week_monday_start(date):
    start_date = pd.to_datetime('2023-07-01')
    days_since_start = (date - start_date).days

    # Adjusting for a Monday start
    offset = (start_date.dayofweek - 0) % 7
    days_since_start += offset

    if days_since_start < 0:
        return 0  # If the date is before the start date, return 0 or handle as needed
    else:
        if days_since_start <= 7:
            return 1
        else:
            week_number = 1 + ((days_since_start - 7) // 7)  # Starting from week 2
            return week_number

# Assuming 'df' is your DataFrame and 'CreatedOn' is your date field
df['CreatedOn'] = pd.to_datetime(df['CreatedOn'], format='%d/%m/%Y %H:%M', errors='coerce')

# Apply the function to calculate the week number
df['FY24 Week'] = df['CreatedOn'].apply(calculate_week_monday_start)

In [47]:
df.sample(5)

,ApplicantId,Filed As,Status,CreatedOn,Source,Feedback Score,Country,Registration #,Registration Type,Registration Expiry,Employment Preference Attributes,Specialty,Seniority,Primary Consultant,FY24 Week
1921,217008,Sajan Sawai Soothar,Transitional,2023-12-25 22:09:00,Organic Search,NaN,Pakistan,NaN,NaN,NaN,Permanent,Medicine - Gastroenterology,Consultant / Specialist,NaN,26
1090,215712,Georgia Hikila,Live,2023-10-24 14:57:00,Referral - Personal referral,NaN,Australia,MED0002569135,General,30/09/2024,Locum,Psych - General Adult,Resident (RMO),Robyn Pascoe,17
1293,215995,Jennifer Yang,Passive,2023-11-06 12:25:00,RANZCOG ASM 2023,NaN,Australia,MED0002557395,General,30/09/2024,Locum,Obstetrics & Gynaecology,Resident (RMO),Dominic Tan,19
1229,215883,Aurora G,Ineligible,2023-10-30 17:01:00,WONCA 2023,NaN,Australia,NaN,NaN,NaN,Permanent,General Practice,General Practitioner,NaN,18
3366,220050,Mostafa Elsawi,Ineligible,2024-03-25 17:15:00,Indeed,NaN,Sudan,NaN,NaN,NaN,Permanent,General Practice,General Practitioner,NaN,39


## Map Team Allocated

In [48]:
# Create a dictionary to map 'Primary Consultant' to 'team allocated'
team_mapping = {
    'Tahnee Love': 'JLD',
    'Craig Picard': 'JLD',
    'Ben Chegwidden': 'JLD',
    'Fiona Jackson': 'JLD',
    'Aimee Skoyles': 'JLD',
    'Mikaila Brooks': 'JLD',
    'Robyn Pascoe': 'Psych',
    'Courtney Lewis': 'Psych',
    'Yasmin Lockey': 'Psych',
    'Caitlin Lingard': 'HS Perm',
    'Jennifer Salinas': 'HS Perm',
    'Ashlea Harvey': 'HS Perm',
    'Chloe Frost': 'HS Perm',
    'EJ Cuaresma': 'GP Perm',
    'Jane Stanke': 'GP Perm',
    'Amber Derby-Davies': 'HS Locum',
    'Jade Camilleri': 'HS Locum',
    'Claudine Zaarour': 'HS Locum',
    'Hannah Kearns': 'HS Locum',
    'Eamon McCurry': 'HS Locum',
    'Nicole Langan': 'HS Locum',
    'Amy Beddall': 'GP Locum',
    'Cory Robertson': 'GP Locum',
    'Charlotte Hellmundt': 'GP Locum',
    'Anna Mullins': 'GP Locum',
    'Jade-Maree Camilleri': 'HS Locum',
    'Xanthia Gardner': 'HS Perm',
    'Charlie Hellmundt': 'GP Locum',
    'Paul Campbell': 'GP Locum',
    'Sharlina Drutschmann': 'GP Locum',
}

# Create the 'team allocated' column based on 'Primary Consultant' using the mapping dictionary
df['Team Allocated'] = df['Primary Consultant'].map(team_mapping)


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3425 entries, 0 to 4342
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ApplicantId                       3425 non-null   int64         
 1   Filed As                          3425 non-null   object        
 2   Status                            3425 non-null   object        
 3   CreatedOn                         3425 non-null   datetime64[ns]
 4   Source                            3425 non-null   object        
 5   Feedback Score                    0 non-null      float64       
 6   Country                           3425 non-null   object        
 7   Registration #                    898 non-null    object        
 8   Registration Type                 896 non-null    object        
 9   Registration Expiry               878 non-null    object        
 10  Employment Preference Attributes  3425 non-null   obj

In [50]:
# Filter for unmapped rows (where 'Primary Consultant' is not null and 'Team Allocated' is missing)
unmapped_rows = df[df['Primary Consultant'].notna() & df['Team Allocated'].isna()]

# Get the unique values of 'Primary Consultant' for unmapped rows
unique_primary_consultants = unmapped_rows['Primary Consultant'].unique()

# Display the unique values
print(unique_primary_consultants)

['Dominic Tan' 'Carole Paterson' 'Alex Graham' 'Wave Info'
 'Lornita Papworth' 'Jecca Bacarisas' 'Tabather Cain' 'Claire Ponsford'
 'Roneeta Chand' 'Nastya Kikteva' 'Max Drakeley' 'Malavika Vaz']


In [51]:
# Non-Australia - Assign 'HS Perm' or 'GP Perm' based on specified conditions
non_australia = (df['Country'] != 'Australia')

# Check if neither 'Specialty' nor 'Seniority' contains 'GP' related phrases
hs_perm_conditions = (
    non_australia &
    ~(
        (df['Specialty'].str.contains('GP|General Practice|General Practitioner', case=False)) | 
        (df['Seniority'].str.contains('GP|General Practice|General Practitioner', case=False))
    )
)

# Update 'HS Perm' for null 'Team Allocated' where the conditions are met
df.loc[hs_perm_conditions & df['Team Allocated'].isnull(), 'Team Allocated'] = 'HS Perm'

# Check for 'GP Perm' based on the same conditions
gp_perm_conditions = (
    non_australia &
    (
        (df['Specialty'].str.contains('GP|General Practice|General Practitioner', case=False)) | 
        (df['Seniority'].str.contains('GP|General Practice|General Practitioner', case=False))
    )
)

# Update 'GP Perm' for null 'Team Allocated' based on specified conditions
df.loc[gp_perm_conditions & df['Team Allocated'].isnull(), 'Team Allocated'] = 'GP Perm'


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3425 entries, 0 to 4342
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ApplicantId                       3425 non-null   int64         
 1   Filed As                          3425 non-null   object        
 2   Status                            3425 non-null   object        
 3   CreatedOn                         3425 non-null   datetime64[ns]
 4   Source                            3425 non-null   object        
 5   Feedback Score                    0 non-null      float64       
 6   Country                           3425 non-null   object        
 7   Registration #                    898 non-null    object        
 8   Registration Type                 896 non-null    object        
 9   Registration Expiry               878 non-null    object        
 10  Employment Preference Attributes  3425 non-null   obj

In [53]:
#Australia Perm
employment_conditions = (
    (df['Country'] == 'Australia') &
    (df['Employment Preference Attributes'].notna()) &
    (~df['Employment Preference Attributes'].str.contains('Locum', case=False, na=False))
)

gp_perm_conditions = (
    employment_conditions &
    (
        (df['Specialty'].str.contains('GP|General Practice|General Practitioner', case=False, na=False)) | 
        (df['Seniority'].str.contains('GP|General Practice|General Practitioner', case=False, na=False))
    )
)

# Set 'GP Perm' for 'Team Allocated' based on conditions
df.loc[gp_perm_conditions & df['Team Allocated'].isnull(), 'Team Allocated'] = 'GP Perm'

# Set 'HS Perm' for remaining null 'Team Allocated' based on conditions
df.loc[employment_conditions & df['Team Allocated'].isnull(), 'Team Allocated'] = 'HS Perm'

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3425 entries, 0 to 4342
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ApplicantId                       3425 non-null   int64         
 1   Filed As                          3425 non-null   object        
 2   Status                            3425 non-null   object        
 3   CreatedOn                         3425 non-null   datetime64[ns]
 4   Source                            3425 non-null   object        
 5   Feedback Score                    0 non-null      float64       
 6   Country                           3425 non-null   object        
 7   Registration #                    898 non-null    object        
 8   Registration Type                 896 non-null    object        
 9   Registration Expiry               878 non-null    object        
 10  Employment Preference Attributes  3425 non-null   obj

In [55]:
#Psych
psych_conditions = (
    (df['Country'] == 'Australia') &
    (df['Specialty'].str.startswith('Psych')) &
    (df['Team Allocated'].isnull())
)

# Set 'Psych' for 'Team Allocated' if conditions are met
df.loc[psych_conditions, 'Team Allocated'] = 'Psych'

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3425 entries, 0 to 4342
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ApplicantId                       3425 non-null   int64         
 1   Filed As                          3425 non-null   object        
 2   Status                            3425 non-null   object        
 3   CreatedOn                         3425 non-null   datetime64[ns]
 4   Source                            3425 non-null   object        
 5   Feedback Score                    0 non-null      float64       
 6   Country                           3425 non-null   object        
 7   Registration #                    898 non-null    object        
 8   Registration Type                 896 non-null    object        
 9   Registration Expiry               878 non-null    object        
 10  Employment Preference Attributes  3425 non-null   obj

In [57]:
#Junior
jld_conditions = (
    df['Team Allocated'].isnull() &
    df['Seniority'].str.contains('resident|registrar|cmo / smo|intern|student', case=False)
)

# Set 'JLD' for 'Team Allocated' where conditions are met
df.loc[jld_conditions, 'Team Allocated'] = 'JLD'

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3425 entries, 0 to 4342
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ApplicantId                       3425 non-null   int64         
 1   Filed As                          3425 non-null   object        
 2   Status                            3425 non-null   object        
 3   CreatedOn                         3425 non-null   datetime64[ns]
 4   Source                            3425 non-null   object        
 5   Feedback Score                    0 non-null      float64       
 6   Country                           3425 non-null   object        
 7   Registration #                    898 non-null    object        
 8   Registration Type                 896 non-null    object        
 9   Registration Expiry               878 non-null    object        
 10  Employment Preference Attributes  3425 non-null   obj

In [59]:
#Australia Locum
# Check if either 'Specialty' or 'Seniority' contains GP-related phrases
gp_locum_conditions = (
    (df['Team Allocated'].isnull()) &
    (df['Specialty'].str.contains('GP|General Practitioner|General Practice', case=False) |
    df['Seniority'].str.contains('GP|General Practitioner|General Practice', case=False))
)

# Set 'GP Locum' for rows with null 'Team Allocated' based on the specified conditions
df.loc[gp_locum_conditions, 'Team Allocated'] = 'GP Locum'

# Set 'HS Locum' for the remaining null 'Team Allocated' rows that don't meet the GP conditions
df.loc[df['Team Allocated'].isnull(), 'Team Allocated'] = 'HS Locum'

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3425 entries, 0 to 4342
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ApplicantId                       3425 non-null   int64         
 1   Filed As                          3425 non-null   object        
 2   Status                            3425 non-null   object        
 3   CreatedOn                         3425 non-null   datetime64[ns]
 4   Source                            3425 non-null   object        
 5   Feedback Score                    0 non-null      float64       
 6   Country                           3425 non-null   object        
 7   Registration #                    898 non-null    object        
 8   Registration Type                 896 non-null    object        
 9   Registration Expiry               878 non-null    object        
 10  Employment Preference Attributes  3425 non-null   obj

In [61]:
#diveregent  careers/ executives
# Condition to check Seniority and Country
seniority_conditions = (
    (df['Seniority'].str.contains('Director|DMS|Health Executive|Divergent Careers|Allied Health|Medical Admin|Scientist', case=False, na=False)) &
    (df['Country'] == 'Australia')
)

# Update 'HS Perm' for the specified conditions
df.loc[seniority_conditions, 'Team Allocated'] = 'HS Perm'

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3425 entries, 0 to 4342
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ApplicantId                       3425 non-null   int64         
 1   Filed As                          3425 non-null   object        
 2   Status                            3425 non-null   object        
 3   CreatedOn                         3425 non-null   datetime64[ns]
 4   Source                            3425 non-null   object        
 5   Feedback Score                    0 non-null      float64       
 6   Country                           3425 non-null   object        
 7   Registration #                    898 non-null    object        
 8   Registration Type                 896 non-null    object        
 9   Registration Expiry               878 non-null    object        
 10  Employment Preference Attributes  3425 non-null   obj

In [63]:
# Check for the specified condition
condition = (df['Seniority'].str.contains('Director|DMS|Health Executive|Divergent Careers|Allied Health|Medical Admin|Scientist', case=False, na=False))

# Get value counts for 'Team Allocated' for rows that satisfy the condition
team_allocated_counts = df.loc[condition, 'Team Allocated'].value_counts()

# Print the value counts
print(team_allocated_counts)

Team Allocated
HS Perm    102
GP Perm     14
Name: count, dtype: int64


In [64]:
seniority_condition = df['Seniority'].str.contains('Director|DMS|Health Executive|Divergent Careers|Allied Health|Medical Admin|Scientist', case=False, na=False)
gp_perm_condition = df['Team Allocated'] == 'GP Perm'

filtered_df = df[seniority_condition & gp_perm_condition]

# Display relevant columns
filtered_df.head()

,ApplicantId,Filed As,Status,CreatedOn,Source,Feedback Score,Country,Registration #,Registration Type,Registration Expiry,Employment Preference Attributes,Specialty,Seniority,Primary Consultant,FY24 Week,Team Allocated
739,215009,Rakesh Sachdeva,Transitional,2023-09-27 22:14:00,Linkedin,NaN,United Kingdom,NaN,NaN,NaN,Permanent,"Paeds - Respiratory Medicine, Paeds - Infectio...","Divergent Careers, Deputy Director of Medical ...",NaN,13,GP Perm
871,215235,Khyber Alam,Passive,2023-10-07 03:01:00,Linkedin,NaN,United Kingdom,NaN,NaN,NaN,"Permanent, Telehealth","GP Telehealth, Medical Administration, General...","Director / Clinical Director, Executive Direct...",NaN,14,GP Perm
1289,215989,Sergio Ortiz-Alinque,Transitional,2023-11-06 11:52:00,Paid Social Lead Gen Linkedin,NaN,Spain,NaN,NaN,NaN,Permanent,General Practice,Divergent Careers,Jane Stanke,19,GP Perm
1680,216643,Kudzai Machingauta,Ineligible,2023-12-07 04:53:00,Organic Search,NaN,Zimbabwe,NaN,NaN,NaN,Permanent,"Public Health, GP Anaesthetics",Divergent Careers,NaN,23,GP Perm
2702,217910,Munzir Hamid,Ineligible,2024-01-18 14:25:00,Linkedin,NaN,Oman,NaN,NaN,NaN,Permanent,"GP Emergency, Emergency Medicine (A&E), Genera...","Resident (RMO), Health Executive",NaN,29,GP Perm





If starts with unknown, other and followed by , second argument then change to 2nd argument
Allied Health
Anaesthetics
Divrgent Careers
Emergency
General Manager
General Practice
GP AMS
GP Emergency
GP Obstetrics
GP Sexual Health
GP Skin
GP Surgery
GP Telehealth
GP Urgent Care
ICU
Medical Administration
Medicine - Cardiology
Medicine - Dermatology
Medicine - Gastroenterology
Medicine – General
Medicine - Geriatric
Medicine - Haematology
Medicine - Hepatology
Medicine - Infectious Dis
Medicine - Nephrology
Medicine - Neurology
Medicine - Oncology
Medicine - Palliative Care
Medicine - Rehabilitation
Medicine - Respiratory
Medicine - Rheumatology
Medicine - Sports
Medicine - Stroke
Nuclear Medicine
Obstetrics & Gynaecology
Other
Paediatrics
Pathology
Psychiatry
Public Health
Radiation - Oncology
Radiology
Surgery
Unknown


In [65]:
# List of specified categories with the corrected "Intensive Care"
categories = [
    "Allied Health", "Anaesthetics", "Divergent Careers", "Emergency Medicine", "General Manager", 
    "General Practice", "GP AMS", "GP Emergency", "GP Obstetrics", "GP Sexual Health", "GP Occ Health",
    "GP Skin", "GP Surgery", "GP Telehealth", "GP Urgent Care", "Intensive Care", 
    "Medical Administration", "Medicine - Cardiology", "Medicine - Dermatology", "Medicine - Diabetes",
    "Medicine - Gastroenterology", "Medicine – General", "Medicine - Geriatric", "Medicine - Occupational Health", 
    "Medicine - Haematology", "Medicine - Hepatology", "Medicine - Infectious Dis", "Medicine - Transfusion Medicine",
    "Medicine - Nephrology", "Medicine - Neurology"," Medicine - Occupational Health", "Medicine - Oncology", "Medicine - Immunology",
    "Medicine - Palliative Care", "Medicine - Rehabilitation", "Medicine - Respiratory", "Medicine - Pain",
    "Medicine - Rheumatology", "Medicine - Sports", "Medicine - Stroke", 
    "Nuclear Medicine", "Obstetrics & Gynaecology", "Other", "Paeds", 
    "Path", "Psych", "Public Health", "Radiation - Oncology", 
    "Radiology", "Surgery", "Unknown"
]

# Sort categories by length in descending order
categories.sort(key=len, reverse=True)

# Function to check if a category is in the specialty
def match_category(specialty, categories):
    for category in categories:
        if category in specialty:
            return category
    return 'Other'

# Function to process the Specialty field
def process_specialty(specialty):
    if pd.isna(specialty):
        return 'Unknown'
    
    parts = specialty.split(',')
    first_arg = parts[0].strip()
    
    if (first_arg.startswith('Unknown') or first_arg.startswith('Other')) and len(parts) > 1:
        second_arg = parts[1].strip()
        return match_category(second_arg, categories)
    
    return match_category(first_arg, categories)

# Apply the function to the 'Specialty' column
df['Specialty'] = df['Specialty'].apply(process_specialty)

# Check the result
print(df['Specialty'].value_counts())



Specialty
Medicine – General                 622
General Practice                   570
Surgery                            306
Emergency Medicine                 260
Obstetrics & Gynaecology           223
Unknown                            214
Psych                              182
Anaesthetics                       131
Intensive Care                     112
Paeds                              105
Radiology                           86
Path                                64
Divergent Careers                   50
Public Health                       48
GP Telehealth                       47
Other                               44
GP Emergency                        40
GP AMS                              39
Medicine - Cardiology               31
Medicine - Gastroenterology         23
Medicine - Respiratory              22
Allied Health                       19
GP Skin                             19
Medicine - Oncology                 18
Medical Administration              14
Medicine - Neph

## Save File

In [66]:
# Get the current date
current_date = datetime.datetime.now()

# Calculate the date of the previous Sunday
previous_sunday = current_date - timedelta(days=current_date.weekday() + 1)

# Format the date in the desired format (YYYYMMDD)
previous_sunday_str = previous_sunday.strftime("%d%m%Y")

# Create a folder if it doesn't exist
folder_name = 'cleaned_new_leads'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Construct the file path with the folder and the previous Sunday's date
file_path = os.path.join(folder_name, f'newleads_cleaned_{previous_sunday_str}.csv')

# Export DataFrame to a CSV file
df.to_csv(file_path, index=False, encoding=result['encoding'])

# Print a success message
print(f'File saved as {file_path}')

File saved as cleaned_new_leads\newleads_cleaned_21072024.csv
